In [ ]:
import os, sys
import platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("Working Directory:", os.getcwd())

In [ ]:
import os
#manually added my FRED API key as an enivronment variable
print(bool(os.getenv("FRED_API_KEY")))

In [ ]:
import pandas as pd
from fredapi import Fred
fred = Fred()
# national labor force participation rate (CIVPART)
series = fred.get_series("CIVPART") 
print(series.index.min(), "->", series.index.max())
print("Observations:", series.shape[0])

In [ ]:
from fredapi import Fred
import pandas as pd

fred = Fred()

STATE_NAMES = [
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia",
    "Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
    "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey",
    "New Mexico","New York","North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania","Rhode Island","South Carolina",
    "South Dakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming","District of Columbia"
]

In [ ]:
# State-level official FIPS codes for standardization
STATE_FIPS = {
    "AL": "01","AK": "02","AZ": "04","AR": "05","CA": "06","CO": "08","CT": "09","DE": "10","DC": "11","FL": "12",
    "GA": "13","HI": "15","ID": "16","IL": "17","IN": "18","IA": "19","KS": "20","KY": "21","LA": "22","ME": "23",
    "MD": "24","MA": "25","MI": "26","MN": "27","MS": "28","MO": "29","MT": "30","NE": "31","NV": "32","NH": "33",
    "NJ": "34","NM": "35","NY": "36","NC": "37","ND": "38","OH": "39","OK": "40","OR": "41","PA": "42","RI": "44",
    "SC": "45","SD": "46","TN": "47","TX": "48","UT": "49","VT": "50","VA": "51","WA": "53","WV": "54","WI": "55","WY": "56"
}

#Generate the name of the data series used by FRED based on state abbrevation
def state_lfp_series_id_from_fips(state_abbr: str) -> str:
    """
    Build the seasonally-adjusted Labor Force Participation Rate series ID for a state.
    Pattern: 'LBSSA' + 2-digit FIPS (e.g., MI=26 -> 'LBSSA26').
    """
    fips = STATE_FIPS.get(state_abbr)
    if fips is None:
        raise ValueError(f"No FIPS for state abbr {state_abbr}")
    return f"LBSSA{fips}"

#Pull the series from FRED API, return yearly LFPR average
def get_state_lfp(state_name: str, state_abbr: str, start_year: int = 2010, end_year: int = 2018):
    sid = state_lfp_series_id_from_fips(state_abbr)

    # Constrain the data fetch to appropriate range
    s = fred.get_series(
        sid,
        observation_start=f"{start_year}-01-01",
        observation_end=f"{end_year}-12-31"
    )

    if s is None or s.empty:
        print(f"No data for {state_name} ({state_abbr})")
        return None

    s.index = pd.to_datetime(s.index)
    # compute annual average from monthly
    annual = s.resample("YE").mean().to_frame(name="lfp_sa")
    annual["year"] = annual.index.year
    annual["state_abbr"] = state_abbr
    return annual[["state_abbr", "year", "lfp_sa"]]

#test on my two favorite states..
mi = get_state_lfp("Michigan", "MI")
ma = get_state_lfp("Massachusetts", "MA")
display(mi.tail())
display(ma.tail())

In [ ]:
# building state-level LFP panel (2010–2019)
import time
all_states = []
STATE_ABBRS = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA",
    "HI","ID","IL","IN","IA","KS","KY","LA","ME","MD",
    "MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC",
    "SD","TN","TX","UT","VT","VA","WA","WV","WI","WY","DC"
]
STATE_MAP = list(zip(STATE_NAMES, STATE_ABBRS))

for name, abbr in STATE_MAP:
    try:
        df = get_state_lfp(name, abbr)
        time.sleep(1)
        if df is not None and not df.empty:
            all_states.append(df)
        else:
            print(f"Skipped{name}")
    except Exception as e:
        print(f"Error with {name} ({abbr}):", e)

lfp_panel = pd.concat(all_states, ignore_index=True)
print("Shape:", lfp_panel.shape)
print(lfp_panel.head())

# save to data folder
out_path = "../data/lfp_state_year.csv"
lfp_panel.to_csv(out_path, index= False)
print("Saved file to:", out_path)

In [ ]:
import os
print(os.getcwd())

In [ ]:
# U.S. Census ACS 1-year “S1501: Educational Attainment” data scraping
import pandas as pd
import requests

# map Census state FIPS -> Previous state abbreviation for correct alignment
FIPS_TO_ABBR = {
    "01":"AL","02":"AK","04":"AZ","05":"AR","06":"CA","08":"CO","09":"CT","10":"DE","11":"DC","12":"FL",
    "13":"GA","15":"HI","16":"ID","17":"IL","18":"IN","19":"IA","20":"KS","21":"KY","22":"LA","23":"ME",
    "24":"MD","25":"MA","26":"MI","27":"MN","28":"MS","29":"MO","30":"MT","31":"NE","32":"NV","33":"NH",
    "34":"NJ","35":"NM","36":"NY","37":"NC","38":"ND","39":"OH","40":"OK","41":"OR","42":"PA","44":"RI",
    "45":"SC","46":"SD","47":"TN","48":"TX","49":"UT","50":"VT","51":"VA","53":"WA","54":"WV","55":"WI","56":"WY"
}

def fetch_bachelors_share_year(year: int) -> pd.DataFrame:
    """
    ACS 1-year subject table S1501 (Educational Attainment).
    S1501_C02_015E = Percent age 25+ with Bachelor's degree or higher (estimate).
    """
    import pandas as pd, requests

    url = f"https://api.census.gov/data/{year}/acs/acs1/subject"
    params = {
        "get": "NAME,S1501_C02_015E",
        "for": "state:*"
    }
    r = requests.get(url, params=params, timeout=30)
    print("Requested:", r.url)
    r.raise_for_status()

    data = r.json()
    cols = data[0] # ["NAME","S1501_C02_015E","state"]
    rows = data[1:]
    df = pd.DataFrame(rows, columns=cols)

    # clean up
    df = df.rename(columns={"state": "state_fips"})
    df["bachelors_share"] = pd.to_numeric(df["S1501_C02_015E"], errors="coerce")
    df["state_abbr"] = df["state_fips"].map(FIPS_TO_ABBR)
    df["year"] = year

    out = df[["state_abbr", "year", "bachelors_share"]].dropna(subset=["state_abbr"])
    return out


# --- test for 2015 ---
edu_2015 = fetch_bachelors_share_year(2015)
edu_2015.head()

In [ ]:
#education by state, save to panel
import pandas as pd

years = list(range(2010,2019))
frames = []

for y in years: 
    try: 
        df = fetch_bachelors_share_year(y)
        print(f"Year {y}: {df.shape[0]} rows")
        frames.append(df)
    except Exception as e: 
        print(f"Skipped {y} due to error {e}")
edu_panel = pd.concat(frames, ignore_index= True)
print("Total rows:", edu_panel.shape[0])
print(edu_panel.head())

In [ ]:
lfp_panel["year"].unique()

In [ ]:
# Save ACS education panel (2010–2018)
edu_panel.to_csv("../data/edu_state_year.csv", index=False) 
print("Saved: ../data/edu_state_year.csv")

In [ ]:
#merge the two data sets by month
merged = pd.merge(lfp_panel, edu_panel, on=["state_abbr", "year"], how="inner")
print("Merged shape:", merged.shape)
merged.head()

In [ ]:
#data analysis
# data preview
print("Shape:", merged.shape)
print("\nColumn types:")
print(merged.dtypes)

print("\nMissing values per column:")
print(merged.isna().sum())

print("\nSummary statistics:")
print(merged.describe())

# Check time coverage
print("\nYears:", merged['year'].unique())
print("\nStates:", merged['state_abbr'].nunique())


In [ ]:
merged.to_csv("../data/merged_state_panel.csv", index=False)
print("Merged dataset exported successfully to ../data/merged_state_panel.csv")
